In [ ]:
import torch
import torchvision

import matplotlib.pyplot as plt
import torch
import torchvision

import requests

from torch import nn
from torchvision import transforms

from pathlib import Path

try:
  from torchinfo import summary
except:
  print("[INFO] Couldn't find torchinfo... installing it.")
  !pip install -q torchinfo
  from torchinfo import summary


try:
  from modular import download_image_data, data_setup, train_model, save
except:
  # Get the going_modular scripts
  print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
  !git clone https://github.com/AaronJ59/Projects.git
  !mv Projects/BiteID/modular/modular modular
  !mv Projects/BiteID/models models
  from modular import download_image_data, data_setup, train_model, save
  !rm -rf Projects


[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.
Cloning into 'Projects'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 163 (delta 33), reused 10 (delta 1), pack-reused 54 (from 2)
Receiving objects: 100% (163/163), 109.87 MiB | 56.45 MiB/s, done.
Resolving deltas: 100% (42/42), done.
mv: cannot stat 'Projects/BiteID/models': No such file or directory


In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

from torch import nn

import os
import zipfile

from pathlib import Path

import requests

# Walk through an image classification directory and find out how many files (images)
# are in each subdirectory.
import os

In [ ]:
# Device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:

# Get the image data from github
download_image_data.download_image_data(source="https://github.com/AaronJ59/Projects/raw/refs/heads/main/BiteID/pizza_hamburger_lasagna_sushi_steak.zip",
                            destination="pizza_hamburger_lasagna_sushi_steak")


#Setup train directory and test directory strings

train_dir = "data/pizza_hamburger_lasagna_sushi_steak/train"
test_dir = "data/pizza_hamburger_lasagna_sushi_steak/test"



Writing zip file to data/


## Training effnetb2

In [ ]:
def create_effnetb2_model(num_classes: int,
                          seed:int=42):


  # Import efficientnetb2

  # Get the weights, transforms of the model and make the model
  efficientnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  effnetb2_transform = efficientnetb2_weights.transforms()
  effnetb2 = torchvision.models.efficientnet_b2(weights=efficientnetb2_weights)

  # Freeze the base layers of efficientnetb2
  for param in effnetb2.parameters():
    param.requires_grad = False



  # Change classifier head of effnetb2 to suit 5 classes
  torch.manual_seed(seed)
  effnetb2.classifier = nn.Sequential(
      nn.Dropout(p=0.3, inplace=True),
      nn.Linear(in_features=1408, out_features=num_classes),
  )

  return effnetb2, effnetb2_transform




# Create an instance
effnetb2, effnetb2_transform = create_effnetb2_model(num_classes=5,
                                                           seed=42)


Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 94.7MB/s]


In [ ]:
summary(effnetb2,
        input_size=(1, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 5]               --                   Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 1

In [ ]:
train_dataloader_effnetb2, test_dataloader_effnetb2, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                                 test_dir=test_dir,
                                                                                                 transform=effnetb2_transform,
                                                                                                 batch_size=32)

In [ ]:
### Train effnetb2 model

# get optimizer
optimizer = torch.optim.Adam(params = effnetb2.parameters(),
                             lr=.001)

# Get loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Train effnetb2
torch.manual_seed(42)
torch.cuda.manual_seed(42)


effnetb2_results = train_model.train(model=effnetb2,
                                     train_dataloader=train_dataloader_effnetb2,
                                     test_dataloader=test_dataloader_effnetb2,
                                     loss_fn=loss_fn,
                                     optimizer=optimizer,
                                     device=device,
                                     epochs=10)





Training the model for 10 epochs


 10%|█         | 1/10 [01:26<12:55, 86.12s/it]


Epoch: 1 | train_loss: 1.2944 | train_acc: 0.5847 | test_loss: 0.9812 | test_acc: 0.8199


 20%|██        | 2/10 [02:49<11:15, 84.46s/it]


Epoch: 2 | train_loss: 0.8845 | train_acc: 0.7884 | test_loss: 0.7540 | test_acc: 0.8413


...


Epoch: 8 | train_loss: 0.4483 | train_acc: 0.8793 | test_loss: 0.4522 | test_acc: 0.8547


 90%|█████████ | 9/10 [12:31<01:23, 83.20s/it]


Epoch: 9 | train_loss: 0.4232 | train_acc: 0.8786 | test_loss: 0.4423 | test_acc: 0.8459


100%|██████████| 10/10 [13:54<00:00, 83.44s/it]


Epoch: 10 | train_loss: 0.4451 | train_acc: 0.8531 | test_loss: 0.4247 | test_acc: 0.8537


In [ ]:
save.save_model(model=effnetb2,
                target_dir="models",
                model_name="effnetb2_85_percent_accuracy_10_epochs.pth")


NameError: name 'effnetb2' is not defined

### Training effnetb2 with data augmentation

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

In [ ]:
effnetb2, effnetb2_transform = create_effnetb2_model(num_classes=5, seed=42)

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.transforms.functional import InterpolationMode

effnetb2_transform_augment = transforms.Compose([
    transforms.Resize([288], interpolation=InterpolationMode.BICUBIC),
    transforms.CenterCrop([288]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(p=0.5), # ADDED THIS AND ALL THATS BELOW
    transforms.RandomApply([transforms.RandomRotation(15)], p=0.4),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.3)

])

In [ ]:
train_dataloader_effnetb2, test_dataloader_effnetb2, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                                 test_dir=test_dir,
                                                                                                 transform=effnetb2_transform_augment,
                                                                                                 batch_size=32)

NameError: name 'train_dir' is not defined

In [ ]:
train_data = datasets.ImageFolder(train_dir, transform=effnetb2_transform_augment)
test_data = datasets.ImageFolder(test_dir, transform=effnetb2_transform)

class_names = train_data.classes

train_dataloader_effnetb2 = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=os.cpu_count(), pin_memory=True)
test_dataloader_effnetb2 = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=os.cpu_count(), pin_memory=True)

In [ ]:
# get optimizer
optimizer = torch.optim.Adam(params = effnetb2.parameters(),
                             lr=.001)

# Get loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Train effnetb2
torch.manual_seed(42)
torch.cuda.manual_seed(42)


effnetb2_results = train_model.train(model=effnetb2,
                                     train_dataloader=train_dataloader_effnetb2,
                                     test_dataloader=test_dataloader_effnetb2,
                                     loss_fn=loss_fn,
                                     optimizer=optimizer,
                                     device=device,
                                     epochs=10)

Training the model for 10 epochs


 10%|█         | 1/10 [01:30<13:38, 91.00s/it]


Epoch: 1 | train_loss: 1.3040 | train_acc: 0.5891 | test_loss: 0.9805 | test_acc: 0.8120


 20%|██        | 2/10 [02:58<11:51, 88.94s/it]


Epoch: 2 | train_loss: 0.9153 | train_acc: 0.7714 | test_loss: 0.7514 | test_acc: 0.8439


...


Epoch: 8 | train_loss: 0.5035 | train_acc: 0.8490 | test_loss: 0.4501 | test_acc: 0.8683


 90%|█████████ | 9/10 [13:03<01:26, 86.69s/it]


Epoch: 9 | train_loss: 0.4604 | train_acc: 0.8639 | test_loss: 0.4368 | test_acc: 0.8579


100%|██████████| 10/10 [14:29<00:00, 86.97s/it]


Epoch: 10 | train_loss: 0.5065 | train_acc: 0.8276 | test_loss: 0.4190 | test_acc: 0.8745


In [ ]:
save.save_model(model=effnetb2,
                target_dir="models",
                model_name="effnetb2_87_percent_accuracy_10_epochs.pth")

Saving model to: models/effnetb2_87_percent_accuracy_10_epochs.pth


## Training mobilenetv2

In [ ]:
def create_mobilenetv2_model(num_classes: int,
                             seed: int=42):

  # Import MobileNet V2
  mobilenetv2_weights = torchvision.models.MobileNet_V2_Weights.DEFAULT
  mobilenetv2_transform = mobilenetv2_weights.transforms()
  mobilenetv2 = torchvision.models.mobilenet_v2(weights=mobilenetv2_weights)

  # Freeze the base layers of efficientnetb2
  for param in mobilenetv2.parameters():
    param.requires_grad = False

  torch.manual_seed(42)
  mobilenetv2.classifier = nn.Sequential(
      nn.Dropout(p=0.2, inplace=False),
      nn.Linear(in_features=1280, out_features=5, bias=True)
  )

  return mobilenetv2,mobilenetv2_transform



# Create an instance
mobilenetv2, mobilenetv2_transform = create_mobilenetv2_model(num_classes=5,
                                                              seed=42)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 372MB/s]


In [ ]:
summary(mobilenetv2,
        input_size=(1, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
MobileNetV2 (MobileNetV2)                     [1, 3, 224, 224]     [1, 5]               --                   Partial
├─Sequential (features)                       [1, 3, 224, 224]     [1, 1280, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)               [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                        [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                   [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─ReLU6 (2)                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─InvertedResidual (1)                   [1, 32, 112, 112]    [1, 16, 112, 112]    --                   False
│    │    └─Sequential (conv)                 [1, 32, 112, 112]    [1, 16, 11

In [ ]:
### Get dataloaders for mobilenetv2

train_dataloader_mobilenetv2, test_dataloader_mobilenetv2, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                                       test_dir=test_dir,
                                                                                                       transform=mobilenetv2_transform,
                                                                                                       batch_size=32)

### Train mobilenetv2 for 10 epochs

In [ ]:
### Train mobilenetv2 for 10 epochs


# Optimizer
optimizer = torch.optim.Adam(params=mobilenetv2.parameters(),
                             lr=.001)

# Loss function
loss_fn = torch.nn.CrossEntropyLoss()

torch.manual_seed(42)
torch.cuda.manual_seed(42)

mobilenetv2_results = train_model.train(model=mobilenetv2,
                                        train_dataloader=train_dataloader_mobilenetv2,
                                        test_dataloader=test_dataloader_mobilenetv2,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer,
                                        device=device,
                                        epochs=10)

Training the model for 10 epochs


 10%|█         | 1/10 [00:32<04:48, 32.04s/it]


Epoch: 1 | train_loss: 1.3270 | train_acc: 0.5491 | test_loss: 1.0653 | test_acc: 0.7714


 20%|██        | 2/10 [01:03<04:14, 31.78s/it]


Epoch: 2 | train_loss: 0.9417 | train_acc: 0.7812 | test_loss: 0.8479 | test_acc: 0.7840


...


Epoch: 8 | train_loss: 0.5028 | train_acc: 0.8538 | test_loss: 0.5484 | test_acc: 0.8345


 90%|█████████ | 9/10 [04:43<00:31, 31.54s/it]


Epoch: 9 | train_loss: 0.4733 | train_acc: 0.8505 | test_loss: 0.5186 | test_acc: 0.8381


100%|██████████| 10/10 [05:15<00:00, 31.56s/it]


Epoch: 10 | train_loss: 0.4507 | train_acc: 0.8698 | test_loss: 0.5060 | test_acc: 0.8433


### Training mobilenetv2 for 10 epochs with data augmentation


Let's see if this increases our test accuracy.

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os


In [ ]:
mobilenetv2, mobilenetv2_transform = create_mobilenetv2_model(num_classes=5, seed=42)
mobilenetv2_transform

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [ ]:
from torchvision import transforms
from torchvision.transforms import InterpolationMode

mobilenetv2_transform_custom = transforms.Compose([
    transforms.Resize(size=232, interpolation=InterpolationMode.BILINEAR),  # Resize to 232
    transforms.CenterCrop(size=224),  # Crop to 224x224
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(p=0.5), # ADDED THIS AND ALL THATS BELOW
    transforms.RandomApply([transforms.RandomRotation(15)], p=0.4),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.3)
])

In [ ]:
mobilenetv2_transform_custom

Compose(
    Resize(size=232, interpolation=bilinear, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    RandomHorizontalFlip(p=0.5)
    RandomApply(
    p=0.4
    RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
)
    RandomApply(
    p=0.3
    GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))
)
)

In [ ]:
train_data = datasets.ImageFolder(train_dir, transform=mobilenetv2_transform_custom)
test_data = datasets.ImageFolder(test_dir, transform=mobilenetv2_transform)

class_names = train_data.classes

train_dataloader_mobilenetv2 = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=os.cpu_count(), pin_memory=True)
test_dataloader_mobilenetv2 = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=os.cpu_count(), pin_memory=True)

In [ ]:
# Optimizer
optimizer = torch.optim.Adam(params=mobilenetv2.parameters(),
                             lr=.001)

# Loss function
loss_fn = torch.nn.CrossEntropyLoss()

torch.manual_seed(42)
torch.cuda.manual_seed(42)

mobilenetv2_results = train_model.train(model=mobilenetv2,
                                        train_dataloader=train_dataloader_mobilenetv2,
                                        test_dataloader=test_dataloader_mobilenetv2,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer,
                                        device=device,
                                        epochs=10)

Training the model for 10 epochs


 10%|█         | 1/10 [00:29<04:26, 29.66s/it]


Epoch: 1 | train_loss: 1.3479 | train_acc: 0.5193 | test_loss: 1.0641 | test_acc: 0.7407


 20%|██        | 2/10 [00:58<03:52, 29.04s/it]


Epoch: 2 | train_loss: 0.9831 | train_acc: 0.7509 | test_loss: 0.8522 | test_acc: 0.7902


...


Epoch: 8 | train_loss: 0.5783 | train_acc: 0.8118 | test_loss: 0.5249 | test_acc: 0.8397


 90%|█████████ | 9/10 [04:19<00:28, 28.73s/it]


Epoch: 9 | train_loss: 0.5730 | train_acc: 0.8240 | test_loss: 0.4885 | test_acc: 0.8610


100%|██████████| 10/10 [04:48<00:00, 28.81s/it]


Epoch: 10 | train_loss: 0.5261 | train_acc: 0.8420 | test_loss: 0.4890 | test_acc: 0.8610


In [ ]:
save.save_model(model=mobilenetv2,
                target_dir="models",
                model_name="mobilenetv2_84_percent_accuracy_10_epochs.pth")

## Load in models from github

Load in models from github that we trained previously above.

In [ ]:
# Load in the models

In [ ]:
mobilenetv2, mobilenetv2_transform = create_mobilenetv2_model(num_classes=5)


mobilenetv2.load_state_dict(
    torch.load(
        f"models/mobilenetv2_84_percent_accuracy_10_epochs.pth",
        map_location=torch.device("cpu")
    )
)



<ipython-input-23-1135060d5be9>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(


<All keys matched successfully>

In [ ]:
effnetb2, effnetb2_transform = create_effnetb2_model(num_classes=5)

effnetb2.load_state_dict(
    torch.load(
        f="models/effnetb2_86_percent_accuracy_10_epochs.pth",
        map_location=torch.device("cpu")
    )
)

<ipython-input-24-c12e394a3143>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(


<All keys matched successfully>

## Finding out the speed of the predictions of test images with each model

In [ ]:
test_dir = "data/pizza_hamburger_lasagna_sushi_steak/test"

In [ ]:
### Get all the test image paths

from pathlib import Path

image_paths = list(Path(test_dir).glob("*/*.jpg"))


In [ ]:
import pathlib
import torch

from PIL import Image
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing import List, Dict

def pred_and_store(image_paths: List[pathlib.Path],
                   model: torch.nn.Module,
                   transform: torchvision.transforms,
                   device: str = "cuda" if torch.cuda.is_available() else "cpu") -> List[Dict]:

  pred_list = []

  model.to(device)

  for path in tqdm(image_paths):

    pred_dict = {}

    class_name = path.parent.stem
    pred_dict["class"] = class_name

    pred_dict["path"] = path

    start_time = timer()

    image = Image.open(path)

    # Transform the image, add batch dimension and put image on target device
    image_transformed = transform(image).unsqueeze(dim=0).to(device)

    model.eval()

    with torch.inference_mode():
      pred_logits = model(image_transformed)

      end_time = timer()


      pred_probs = torch.softmax(pred_logits, dim=1)
      pred_label = torch.argmax(pred_probs, dim=1)

      pred_dict["pred_prob"] = round(pred_probs.max().cpu().item(), 4)
      pred_dict["pred_class"] = class_names[pred_label.cpu()]



    pred_dict["pred_time"] = round(end_time - start_time, 4)
    pred_dict["correct_pred"] = class_name == class_names[pred_label.cpu()]

    # Append created dictionary to the list
    pred_list.append(pred_dict)

  return pred_list







### Predicting and timing using effnetb2 first

In [ ]:
effnetb2_pred_list = pred_and_store(image_paths=image_paths,
               model=effnetb2,
               transform=effnetb2_transform,
               device=device)

  0%|          | 0/375 [00:00<?, ?it/s]

In [ ]:
effnetb2_pred_list[:5]

[{'class': 'steak',
  'path': PosixPath('data/pizza_hamburger_lasagna_sushi_steak/test/steak/1523026.jpg'),
  'pred_prob': 0.6082,
  'pred_class': 'steak',
  'pred_time': 0.0273,
  'correct_pred': True},
 {'class': 'steak',
  'path': PosixPath('data/pizza_hamburger_lasagna_sushi_steak/test/steak/970916.jpg'),
  'pred_prob': 0.6691,
  'pred_class': 'steak',
  'pred_time': 0.025,
  'correct_pred': True},
 {'class': 'steak',
  'path': PosixPath('data/pizza_hamburger_lasagna_sushi_steak/test/steak/476333.jpg'),
  'pred_prob': 0.9803,
  'pred_class': 'steak',
  'pred_time': 0.0198,
  'correct_pred': True},
 {'class': 'steak',
  'path': PosixPath('data/pizza_hamburger_lasagna_sushi_steak/test/steak/2981911.jpg'),
  'pred_prob': 0.6474,
  'pred_class': 'sushi',
  'pred_time': 0.0233,
  'correct_pred': False},
 {'class': 'steak',
  'path': PosixPath('data/pizza_hamburger_lasagna_sushi_steak/test/steak/1379840.jpg'),
  'pred_prob': 0.7569,
  'pred_class': 'steak',
  'pred_time': 0.0214,
  'corr

In [ ]:
import pandas as pd
effnetb2_pred_df = pd.DataFrame(effnetb2_pred_list)

effnetb2_pred_df.head()

,class,path,pred_prob,pred_class,pred_time,correct_pred
0,steak,data/pizza_hamburger_lasagna_sushi_steak/test/...,0.9603,steak,0.0703,True
1,steak,data/pizza_hamburger_lasagna_sushi_steak/test/...,0.9375,steak,0.0531,True
2,steak,data/pizza_hamburger_lasagna_sushi_steak/test/...,0.8846,steak,0.0520,True
3,steak,data/pizza_hamburger_lasagna_sushi_steak/test/...,0.7471,steak,0.0516,True
4,steak,data/pizza_hamburger_lasagna_sushi_steak/test/...,0.4812,steak,0.0532,True


In [ ]:
# Check number of correct predictions
effnetb2_pred_df.correct_pred.value_counts()

,count
correct_pred,
True,328
False,47


In [ ]:
effnetb2_average_prediction_time = round(effnetb2_pred_df.pred_time.mean(), 4)
print(f"Average time per prediction for mobilenetv2 is {effnetb2_average_prediction_time} seconds")

Average time per prediction for mobilenetv2 is 0.0468 seconds


### Predict and time using mobilenetv2

In [ ]:
import pandas as pd
mobilenetv2_pred_list = pred_and_store(image_paths=image_paths,
                                       model=mobilenetv2,
                                       transform=mobilenetv2_transform,
                                       device=device)

  0%|          | 0/375 [00:00<?, ?it/s]

In [ ]:
mobilenetv2_pred_df = pd.DataFrame(mobilenetv2_pred_list)

mobilenetv2_pred_df.head()

,class,path,pred_prob,pred_class,pred_time,correct_pred
0,pizza,data/pizza_hamburger_lasagna_sushi_steak/test/...,0.9348,pizza,0.0227,True
1,pizza,data/pizza_hamburger_lasagna_sushi_steak/test/...,0.8979,pizza,0.0209,True
2,pizza,data/pizza_hamburger_lasagna_sushi_steak/test/...,0.5930,lasagna,0.0198,False
3,pizza,data/pizza_hamburger_lasagna_sushi_steak/test/...,0.6473,pizza,0.0209,True
4,pizza,data/pizza_hamburger_lasagna_sushi_steak/test/...,0.3118,sushi,0.0205,False


In [ ]:
mobilenetv2_pred_df.correct_pred.value_counts()

,count
correct_pred,
True,322
False,53


mobilenetv2 is performing at an 86% accuracy/correct rate.

In [ ]:
mobilenetv2_average_prediction_time = round(mobilenetv2_pred_df.pred_time.mean(), 4)
print(f"Average time per prediction for mobilenetv2 is {mobilenetv2_average_prediction_time} seconds")

Average time per prediction for mobilenetv2 is 0.0238 seconds


## Training effnetb2 with a larger dataset

Uploading the zip file of the dataset through my local computer

In [ ]:
from zipfile import ZipFile

directory_name = "data/large_pizza_hamburger_lasagna_sushi_steak"

try:
  os.mkdir("data")
  os.mkdir(directory_name)
  print(f"Directory {directory_name} created")
except FileExistsError:
  print(f"Directory already exists")


with zipfile.ZipFile('large_pizza_hamburger_lasagna_sushi_steak.zip', 'r') as zip_ref:
  zip_ref.extractall(directory_name)

Directory data/large_pizza_hamburger_lasagna_sushi_steak created


### Training with effnetb2 with data augmentation and large dataset

In [ ]:
train_dir = "data/large_pizza_hamburger_lasagna_sushi_steak/train"
test_dir = "data/large_pizza_hamburger_lasagna_sushi_steak/test"

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.transforms.functional import InterpolationMode

effnetb2_transform_augment = transforms.Compose([
    transforms.Resize([288], interpolation=InterpolationMode.BICUBIC),
    transforms.CenterCrop([288]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(p=0.5), # ADDED THIS AND ALL THATS BELOW
    transforms.RandomApply([transforms.RandomRotation(15)], p=0.4),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.3)

])

In [ ]:
train_data = datasets.ImageFolder(train_dir, transform=effnetb2_transform_augment)
test_data = datasets.ImageFolder(test_dir, transform=effnetb2_transform)

class_names = train_data.classes

large_train_dataloader_effnetb2 = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=os.cpu_count(), pin_memory=True)
large_test_dataloader_effnetb2 = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=os.cpu_count(), pin_memory=True)

In [ ]:
class_names

In [ ]:
effnetb2, effnetb2_transform = create_effnetb2_model(num_classes=5, seed=42)


In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(params=effnetb2.parameters(),
                             lr=.001)


torch.manual_seed(42)
torch.cuda.manual_seed(42)

effnetb2_large_results = train_model.train(model=effnetb2,
                                           train_dataloader=large_train_dataloader_effnetb2,
                                           test_dataloader=large_test_dataloader_effnetb2,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device,
                                           epochs=10)

Training the model for 10 epochs


 10%|█         | 1/10 [03:53<34:59, 233.27s/it]


Epoch: 1 | train_loss: 1.1079 | train_acc: 0.6657 | test_loss: 0.6861 | test_acc: 0.8681


 20%|██        | 2/10 [07:39<30:34, 229.35s/it]


Epoch: 2 | train_loss: 0.7290 | train_acc: 0.7805 | test_loss: 0.5197 | test_acc: 0.8754


...


Epoch: 8 | train_loss: 0.4610 | train_acc: 0.8509 | test_loss: 0.3426 | test_acc: 0.9057


 90%|█████████ | 9/10 [34:10<03:48, 228.38s/it]


Epoch: 9 | train_loss: 0.4561 | train_acc: 0.8366 | test_loss: 0.3461 | test_acc: 0.9068


100%|██████████| 10/10 [37:58<00:00, 227.90s/it]


Epoch: 10 | train_loss: 0.4574 | train_acc: 0.8385 | test_loss: 0.3352 | test_acc: 0.9037


In [ ]:
save.save_model(model=effnetb2,
                target_dir="models",
                model_name="large_dataset_effnetb2_90_percent_accuracy_10_epochs.pth")

Saving model to: models/large_dataset_effnetb2_90_percent_accuracy_10_epochs.pth


In [ ]:
effnetb2_pred_list = pred_and_store(image_paths=image_paths,
               model=effnetb2,
               transform=effnetb2_transform,
               device=device)

  0%|          | 0/988 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
effnetb2_pred_df = pd.DataFrame(effnetb2_pred_list)

effnetb2_pred_df.head()

In [ ]:
effnetb2_pred_df.correct_pred.value_counts()

,count
correct_pred,
True,893
False,95


# Everything from here down below is for BiteIDv2

## Training effnetb2 on a 80-class food dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.makedirs("data/food80_multiclass_dataset")

!unzip  /content/drive/MyDrive/food80_multiclass_dataset.zip -d /content/data/food80_multiclass_dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/food80_multiclass_dataset/test/fish_and_chips/1784894.jpg  
  inflating: /content/data/food80_multiclass_dataset/test/fish_and_chips/3860849.jpg  
  inflating: /content/data/food80_multiclass_dataset/test/fish_and_chips/2799763.jpg  
  inflating: /content/data/food80_multiclass_dataset/test/fish_and_chips/3388814.jpg  
  inflating: /content/data/food80_multiclass_dataset/test/fish_and_chips/1662738.jpg  
  inflating: /content/data/food80_multiclass_dataset/test/fish_and_chips/3912593.jpg  
  inflating: /content/data/food80_multiclass_dataset/test/fish_and_chips/3909172.jpg  
  inflating: /content/data/food80_multiclass_dataset/test/fish_and_chips/4128.jpg  
  inflating: /content/data/food80_multiclass_dataset/test/fish_and_chips/2497717.jpg  
  inflating: /content/data/food80_multiclass_dataset/test/fish_and_chips/1830225.jpg  
  inflating: /content/data/food80_multiclass_dataset/test/fish_and_chips/2065591.j

In [ ]:
data_dir = Path("data")

food_dir = data_dir / "food80_multiclass_dataset"
train_dir = food_dir / "train"
test_dir = food_dir / "test"

In [ ]:
effnetb2, effnetb2_transform = create_effnetb2_model(num_classes=80)

In [ ]:
train_transform = torchvision.transforms.Compose([
    torchvision.transforms.TrivialAugmentWide(),
    effnetb2_transform
])

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

NUM_WORKERS = os.cpu_count()

train_data = datasets.ImageFolder(train_dir, transform=train_transform)
test_data = datasets.ImageFolder(test_dir, transform=effnetb2_transform)

class_names = train_data.classes

train_dataloader_effnetb2 = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_dataloader_effnetb2 = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

In [ ]:
optimizer = torch.optim.Adam(params=effnetb2.parameters(), lr=.001)

loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=.1)

effnetb2_food80_results = train_model.train(model=effnetb2,
                                            train_dataloader=train_dataloader_effnetb2,
                                            test_dataloader=test_dataloader_effnetb2,
                                            loss_fn=loss_fn,
                                            optimizer=optimizer,
                                            device=device,
                                            epochs=10)

Training the model for 10 epochs


 10%|█         | 1/10 [03:02<27:19, 182.18s/it]


Epoch: 1 | train_loss: 2.9458 | train_acc: 0.3978 | test_loss: 2.1355 | test_acc: 0.6076


 20%|██        | 2/10 [06:06<24:29, 183.73s/it]


Epoch: 2 | train_loss: 2.5091 | train_acc: 0.4936 | test_loss: 2.0393 | test_acc: 0.6276


...


Epoch: 8 | train_loss: 2.3882 | train_acc: 0.5316 | test_loss: 1.9466 | test_acc: 0.6597


 90%|█████████ | 9/10 [27:42<03:05, 185.09s/it]


Epoch: 9 | train_loss: 2.3776 | train_acc: 0.5331 | test_loss: 1.9420 | test_acc: 0.6577


100%|██████████| 10/10 [30:47<00:00, 184.79s/it]


Epoch: 10 | train_loss: 2.3804 | train_acc: 0.5317 | test_loss: 1.9464 | test_acc: 0.6575


In [ ]:
###  Test accuracy stayed the same even though I increased the data size from 40% to 70%

### Test if results are true

In [ ]:
from pathlib import Path

image_paths = list(Path(test_dir).glob("*/*.jpg"))

In [ ]:


effnetb2_pred_list = pred_and_store(image_paths=image_paths,
               model=effnetb2,
               transform=effnetb2_transform,
               device=device)

  0%|          | 0/14000 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
effnetb2_pred_df = pd.DataFrame(effnetb2_pred_list)

effnetb2_pred_df.head()

,class,path,pred_prob,pred_class,pred_time,correct_pred
0,frozen_yogurt,data/food80_multiclass_dataset/test/frozen_yog...,0.6834,frozen_yogurt,0.0259,True
1,frozen_yogurt,data/food80_multiclass_dataset/test/frozen_yog...,0.3497,frozen_yogurt,0.0206,True
2,frozen_yogurt,data/food80_multiclass_dataset/test/frozen_yog...,0.6723,frozen_yogurt,0.0203,True
3,frozen_yogurt,data/food80_multiclass_dataset/test/frozen_yog...,0.7765,frozen_yogurt,0.0209,True
4,frozen_yogurt,data/food80_multiclass_dataset/test/frozen_yog...,0.9894,frozen_yogurt,0.0182,True


In [ ]:
effnetb2_pred_df.correct_pred.value_counts()

,count
correct_pred,
True,9203
False,4797


In [ ]:
save.save_model(model=effnetb2, target_dir="models", model_name="effnetb2_food80_multiclass_model_65_percent_acc.pth")

Saving model to: models/effnetb2_food80_multiclass_model_65_percent_acc.pth
